# qaoa for max-cut experiments

trying out QAOA on the max-cut problem. gonna test different circuit depths and graph types to see how well it works.

what this notebook does:
1. run qaoa on a small example graph
2. benchmark across different graph sizes and depths
3. compare performance on different graph structures
4. look at measurement distributions

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path

from src.qaoa_solver import QAOAMaxCutSolver
from src.maxcut_utils import (
    generate_random_graph, generate_regular_graph, generate_cycle_graph,
    brute_force_maxcut, compute_cut_value
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True)

print('ready to go')

## 1. small example first

lets start with a small graph and see what the optimal cut looks like. then we'll throw qaoa at it.

In [ ]:
# make a small random graph
G_example = generate_random_graph(n_nodes=5, edge_prob=0.6, seed=42)

exact_cut, exact_partition = brute_force_maxcut(G_example)
print(f"nodes: {G_example.number_of_nodes()}, edges: {G_example.number_of_edges()}")
print(f"optimal cut: {exact_cut}")
print(f"optimal partition: {exact_partition}")

# draw it
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
pos = nx.spring_layout(G_example, seed=42)

# plain graph
nx.draw(G_example, pos, ax=axes[0], with_labels=True, node_color='lightblue',
        node_size=500, font_size=14, edge_color='gray', width=2)
axes[0].set_title('Graph')

# graph with partition shown
colors = ['#FF6B6B' if b == 0 else '#4ECDC4' for b in exact_partition]
cut_edges = []
non_cut_edges = []
nodes = list(G_example.nodes())
for u, v in G_example.edges():
    i, j = nodes.index(u), nodes.index(v)
    if exact_partition[i] != exact_partition[j]:
        cut_edges.append((u, v))
    else:
        non_cut_edges.append((u, v))

nx.draw_networkx_nodes(G_example, pos, ax=axes[1], node_color=colors, node_size=500)
nx.draw_networkx_labels(G_example, pos, ax=axes[1], font_size=14)
nx.draw_networkx_edges(G_example, pos, edgelist=non_cut_edges, ax=axes[1],
                       edge_color='lightgray', width=1.5)
nx.draw_networkx_edges(G_example, pos, edgelist=cut_edges, ax=axes[1],
                       edge_color='red', width=2.5, style='dashed')
axes[1].set_title(f'Optimal Cut (value = {exact_cut})')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'example_graph.png', dpi=150, bbox_inches='tight')
plt.show()
print("red dashed = cut edges")

## 2. running qaoa at different depths

now lets see how qaoa does on this graph. trying p=1 through p=5. the approx ratio tells us how close to optimal we get (1.0 = perfect).

In [ ]:
# try different depths
p_values = [1, 2, 3, 4, 5]
results_by_p = {}

for p in p_values:
    solver = QAOAMaxCutSolver(G_example, p=p, seed=42)
    result = solver.solve(maxiter=300)
    results_by_p[p] = result
    print(f"p={p}: cut={result['best_cut_value']}/{result['exact_cut_value']}, "
          f"approx_ratio={result['approx_ratio']:.4f}, "
          f"partition={result['best_partition']}")

In [ ]:
# convergence curves
fig, ax = plt.subplots(figsize=(10, 6))

for idx, p in enumerate(p_values):
    history = results_by_p[p]['cost_history']
    cut_values = [-c for c in history]
    ax.plot(range(len(cut_values)), cut_values, label=f'p={p}', alpha=0.8)

ax.axhline(y=exact_cut, color='red', linestyle='--', label=f'exact ({exact_cut})', alpha=0.7)
ax.set_xlabel('Iteration')
ax.set_ylabel('Expected Cut Value')
ax.set_title('Convergence at different depths')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'convergence_by_depth.png', dpi=150)
plt.show()

## 3. scaling - bigger graphs

ok so it works on small graphs. the real question is how does it scale? sweeping from 3 to 10 nodes with p=1 to 5. running multiple trials per setting for error bars.

this takes a few minutes btw

In [ ]:
# benchmark across sizes and depths
node_range = range(3, 11)
p_range = range(1, 6)
n_trials = 5

benchmark_results = {}

for n in node_range:
    for p in p_range:
        ratios = []
        for trial in range(n_trials):
            seed = trial * 100 + n * 10 + p
            G = generate_random_graph(n, edge_prob=0.5, seed=seed)
            
            if not nx.is_connected(G):
                continue
            
            solver = QAOAMaxCutSolver(G, p=p, seed=seed)
            result = solver.solve(maxiter=200)
            ratios.append(result['approx_ratio'])
        
        if ratios:
            benchmark_results[(n, p)] = {
                'mean': np.mean(ratios),
                'std': np.std(ratios),
            }
    print(f"done with n={n}")

print("\nbenchmark done!")

In [ ]:
# plot approx ratio vs depth
fig, ax = plt.subplots(figsize=(10, 6))

node_counts = sorted(set(n for n, p in benchmark_results.keys()))
p_vals = sorted(set(p for n, p in benchmark_results.keys()))

for idx, n in enumerate(node_counts):
    means = [benchmark_results[(n, p)]['mean'] for p in p_vals if (n, p) in benchmark_results]
    stds = [benchmark_results[(n, p)]['std'] for p in p_vals if (n, p) in benchmark_results]
    valid_p = [p for p in p_vals if (n, p) in benchmark_results]
    
    ax.errorbar(valid_p, means, yerr=stds, marker='o', label=f'n={n}', capsize=3)

ax.set_xlabel('QAOA Depth (p)')
ax.set_ylabel('Approximation Ratio')
ax.set_title('Approx Ratio vs Depth')
ax.legend(title='nodes', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_ylim(0.5, 1.05)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'approx_ratio_vs_p.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# same data but plotted the other way - ratio vs graph size
fig, ax = plt.subplots(figsize=(10, 6))

for idx, p in enumerate(p_vals):
    means = [benchmark_results[(n, p)]['mean'] for n in node_counts if (n, p) in benchmark_results]
    stds = [benchmark_results[(n, p)]['std'] for n in node_counts if (n, p) in benchmark_results]
    valid_n = [n for n in node_counts if (n, p) in benchmark_results]
    
    ax.errorbar(valid_n, means, yerr=stds, marker='s', label=f'p={p}', capsize=3)

ax.set_xlabel('Number of Nodes')
ax.set_ylabel('Approximation Ratio')
ax.set_title('Performance vs Graph Size')
ax.legend(title='depth')
ax.set_ylim(0.5, 1.05)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'approx_ratio_vs_nodes.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. graph type comparison

different graph structures might be easier/harder for qaoa. testing erdos-renyi (random), regular (d=3), and cycle graphs.

cycle is nice because we know the exact answer analytically - for even n its n edges cut.

In [ ]:
# compare graph types
n_compare = 6
n_trials_gt = 3

graph_results = {}

# erdos-renyi
graph_results['Erdos-Renyi'] = {}
for p in range(1, 6):
    ratios = []
    for t in range(n_trials_gt):
        G = generate_random_graph(n_compare, 0.5, seed=t*50+p)
        solver = QAOAMaxCutSolver(G, p=p, seed=t*50+p)
        res = solver.solve(maxiter=200)
        ratios.append(res['approx_ratio'])
    graph_results['Erdos-Renyi'][p] = {'mean': np.mean(ratios), 'std': np.std(ratios)}

# regular graph
graph_results['Regular (d=3)'] = {}
for p in range(1, 6):
    ratios = []
    for t in range(n_trials_gt):
        G = generate_regular_graph(n_compare, 3, seed=t*50+p)
        solver = QAOAMaxCutSolver(G, p=p, seed=t*50+p)
        res = solver.solve(maxiter=200)
        ratios.append(res['approx_ratio'])
    graph_results['Regular (d=3)'][p] = {'mean': np.mean(ratios), 'std': np.std(ratios)}

# cycle
graph_results['Cycle'] = {}
for p in range(1, 6):
    ratios = []
    for t in range(n_trials_gt):
        G = generate_cycle_graph(n_compare)
        solver = QAOAMaxCutSolver(G, p=p, seed=t*50+p)
        res = solver.solve(maxiter=200)
        ratios.append(res['approx_ratio'])
    graph_results['Cycle'][p] = {'mean': np.mean(ratios), 'std': np.std(ratios)}

print("done with graph type comparison")

In [ ]:
# plot graph type comparison
fig, ax = plt.subplots(figsize=(10, 6))

for gtype, p_data in graph_results.items():
    p_list = sorted(p_data.keys())
    means = [p_data[p]['mean'] for p in p_list]
    stds = [p_data[p]['std'] for p in p_list]
    ax.errorbar(p_list, means, yerr=stds, marker='o', label=gtype, capsize=3)

ax.set_xlabel('QAOA Depth (p)')
ax.set_ylabel('Approximation Ratio')
ax.set_title(f'Graph Type Comparison (n={n_compare})')
ax.legend()
ax.set_ylim(0.6, 1.05)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'graph_type_comparison.png', dpi=150)
plt.show()

## 5. measurement distributions

lets look at what the qaoa circuit actually outputs. comparing p=1 (shallow) vs p=5 (deep) - at higher p the distribution should be more concentrated on good solutions.

In [ ]:
# measurement histogram - p=1 vs p=5
G_hist = generate_random_graph(5, 0.6, seed=42)
exact_val, _ = brute_force_maxcut(G_hist)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, p in enumerate([1, 5]):
    solver = QAOAMaxCutSolver(G_hist, p=p, shots=8192, seed=42)
    result = solver.solve(maxiter=300)
    counts = result['measurement_counts']
    
    # sort bitstrings by cut value
    bitstrings = sorted(counts.keys(), key=lambda b: compute_cut_value(G_hist, [int(x) for x in b[::-1]]))
    cut_vals = [compute_cut_value(G_hist, [int(x) for x in b[::-1]]) for b in bitstrings]
    probs = [counts[b] / 8192 for b in bitstrings]
    
    # green = optimal, yellow = close, red = bad
    colors_bar = ['#4CAF50' if cv == exact_val else '#FFC107' if cv >= exact_val - 1 else '#F44336'
                  for cv in cut_vals]
    
    axes[idx].bar(range(len(bitstrings)), probs, color=colors_bar)
    axes[idx].set_xlabel('Bitstring (sorted by cut value)')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_title(f'p={p}')
    axes[idx].set_xticks(range(len(bitstrings)))
    axes[idx].set_xticklabels([f'{b}\n({cv})' for b, cv in zip(bitstrings, cut_vals)],
                              rotation=90, fontsize=6)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'measurement_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print("green=optimal, yellow=close, red=bad")

## takeaways

- more layers (higher p) = better results, but diminishing returns after p=3 for small graphs
- performance drops as graphs get bigger which makes sense
- structured graphs like cycles are easier for qaoa than random ones
- at higher p the output distribution concentrates on good solutions which is cool

would be interesting to try this on actual quantum hardware next but noise would probably mess things up. TODO for later maybe.